In [1]:
from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

from concurrent import futures
import logging
import math
import time
from PIL import Image

import grpc
import wizard_system_pb2
import wizard_system_pb2_grpc

ModuleNotFoundError: No module named 'wizard_system_pb2'

# Server Source

In [ ]:
def resize_image_from_bytes(pre_image, original_size: tuple, target_size: tuple):
    img = np.frombuffer(pre_image, dtype=np.float32).reshape(original_size + (3,))
    img = image.array_to_img(img)
    return img.resize(target_size) 

def predict(model, image):
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)

    preds = model.predict(image).argmax(axis=1)
    print('Predicted:', preds)
    return preds

class WizardSystemServicer(wizard_system_pb2_grpc.WizardServiceServicer):
    
    def __init__(self):
        self.model = keras.models.load_model('model/model_classification_20220116')
        pass
        
    def PostMagicImageRaw(self, request, context):
        context.set_code(grpc.StatusCode.OK)
        resized_image = resize_image_from_bytes(request.imageData, (request.height, request.width), (299, 299))
        preds = predict(self.model, resized_image)
        return wizard_system_pb2.Magic(type=str(preds[0]))

    def PostMagicImagePng(self, request, context):
        context.set_code(grpc.StatusCode.OK)
        context.set_details('Method not implemented!')
        raise NotImplementedError('Method not implemented!')


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    wizard_system_pb2_grpc.add_WizardServiceServicer_to_server(WizardSystemServicer(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    logging.basicConfig()
    serve()
    

# Client

In [9]:
from __future__ import print_function

from tensorflow.keras.preprocessing import image as KerasImage
import logging
import random
import numpy as np

import grpc
from wizard.proto import wizard_api_pb2 as Api
from wizard.proto import wizard_api_pb2_grpc as ApiGrpc


In [15]:
def run():
    # NOTE(gRPC Python Team): .close() is possible on a channel and should be
    # used in circumstances in which the with statement does not fit the needs
    # of the code.
    with grpc.insecure_channel('localhost:50051') as channel:
        stub = ApiGrpc.WizardServiceStub(channel)

        img_path = './test_data/triangle.png'
        img = KerasImage.load_img(img_path, target_size=(300, 300))
        x = KerasImage.img_to_array(img) 

        image = Api.Image(width=300, height=300, imageData=x.astype(np.byte).tobytes())
        result = stub.PostMagicImageRaw(image)
        print("-------------- GetFeature --------------")
        print(str(result))


if __name__ == '__main__':
    logging.basicConfig()
    run()

-------------- GetFeature --------------
type: "square"



In [10]:
img_path = './test_data/triangle.png'
img = KerasImage.load_img(img_path, target_size=(300, 300))
x = KerasImage.img_to_array(img)
x.astype(np.byte).tobytes()
x.astype(np.byte).tobytes()
print(len(x.tobytes()))
np.frombuffer(x.tobytes(), dtype=np.float32).reshape((300,300,3))


1080000


array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [5]:
from os import listdir
from os.path import isfile, join

def all_filename_from(directory: str):
    return [f for f in listdir(directory) if isfile(join(directory, f))]


In [7]:
all_filename_from('./train_data')

[]

# Test Model

In [10]:
import manage
from manage import WizardManager
from wizard import *

from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image as KerasImage

In [14]:
%load_ext autoreload
%autoreload 2

from manage import WizardManager

In [15]:
manager = WizardManager()
classifier = MagicClassifier(manager.model_path)

In [16]:
filenames = manager.all_filename_from(manager.test_data_path)
loaded_image = manager.load_image(manager.test_data_path + filenames[0])

In [28]:
img = KerasImage.load_img(manager.test_data_path + filenames[0], target_size=(299, 299))
arr_img = KerasImage.img_to_array(img)

loaded_image = arr_img.astype(np.float32).reshape((299, 299, 3))

In [29]:
image = np.expand_dims(loaded_image, axis=0)
image = preprocess_input(image)

isValidate = False
preds = classifier.model.predict(image)

In [30]:
preds

array([[7.7850935e-05, 5.5265285e-02, 3.9012154e-04, 9.4381118e-01,
        4.5560490e-04]], dtype=float32)

# Example codes

In [ ]:

class RouteGuideServicer(wizard_system_pb2_grpc.RouteGuideServicer):
    """Provides methods that implement functionality of route guide server."""

    def __init__(self):
        self.db = route_guide_resources.read_route_guide_database()

    def GetFeature(self, request, context):
        feature = get_feature(self.db, request)
        if feature is None:
            return wizard_system_pb2.Feature(name="", location=request)
        else:
            return feature

    def ListFeatures(self, request, context):
        left = min(request.lo.longitude, request.hi.longitude)
        right = max(request.lo.longitude, request.hi.longitude)
        top = max(request.lo.latitude, request.hi.latitude)
        bottom = min(request.lo.latitude, request.hi.latitude)
        for feature in self.db:
            if (feature.location.longitude >= left and
                    feature.location.longitude <= right and
                    feature.location.latitude >= bottom and
                    feature.location.latitude <= top):
                yield feature

    def RecordRoute(self, request_iterator, context):
        point_count = 0
        feature_count = 0
        distance = 0.0
        prev_point = None

        start_time = time.time()
        for point in request_iterator:
            point_count += 1
            if get_feature(self.db, point):
                feature_count += 1
            if prev_point:
                distance += get_distance(prev_point, point)
            prev_point = point

        elapsed_time = time.time() - start_time
        return wizard_system_pb2.RouteSummary(point_count=point_count,
                                            feature_count=feature_count,
                                            distance=int(distance),
                                            elapsed_time=int(elapsed_time))

    def RouteChat(self, request_iterator, context):
        prev_notes = []
        for new_note in request_iterator:
            for prev_note in prev_notes:
                if prev_note.location == new_note.location:
                    yield prev_note
            prev_notes.append(new_note)


In [ ]:
from concurrent import futures
import logging
import math
import time

def get_feature(feature_db, point):
    """Returns Feature at given location or None."""
    for feature in feature_db:
        if feature.location == point:
            return feature
    return None


def get_distance(start, end):
    """Distance between two points."""
    coord_factor = 10000000.0
    lat_1 = start.latitude / coord_factor
    lat_2 = end.latitude / coord_factor
    lon_1 = start.longitude / coord_factor
    lon_2 = end.longitude / coord_factor
    lat_rad_1 = math.radians(lat_1)
    lat_rad_2 = math.radians(lat_2)
    delta_lat_rad = math.radians(lat_2 - lat_1)
    delta_lon_rad = math.radians(lon_2 - lon_1)

    # Formula is based on http://mathforum.org/library/drmath/view/51879.html
    a = (pow(math.sin(delta_lat_rad / 2), 2) +
         (math.cos(lat_rad_1) * math.cos(lat_rad_2) *
          pow(math.sin(delta_lon_rad / 2), 2)))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371000
    # metres
    return R * c


class RouteGuideServicer(route_guide_pb2_grpc.RouteGuideServicer):
    """Provides methods that implement functionality of route guide server."""

    def __init__(self):
        self.db = route_guide_resources.read_route_guide_database()

    def GetFeature(self, request, context):
        feature = get_feature(self.db, request)
        if feature is None:
            return route_guide_pb2.Feature(name="", location=request)
        else:
            return feature

    def ListFeatures(self, request, context):
        left = min(request.lo.longitude, request.hi.longitude)
        right = max(request.lo.longitude, request.hi.longitude)
        top = max(request.lo.latitude, request.hi.latitude)
        bottom = min(request.lo.latitude, request.hi.latitude)
        for feature in self.db:
            if (feature.location.longitude >= left and
                    feature.location.longitude <= right and
                    feature.location.latitude >= bottom and
                    feature.location.latitude <= top):
                yield feature

    def RecordRoute(self, request_iterator, context):
        point_count = 0
        feature_count = 0
        distance = 0.0
        prev_point = None

        start_time = time.time()
        for point in request_iterator:
            point_count += 1
            if get_feature(self.db, point):
                feature_count += 1
            if prev_point:
                distance += get_distance(prev_point, point)
            prev_point = point

        elapsed_time = time.time() - start_time
        return route_guide_pb2.RouteSummary(point_count=point_count,
                                            feature_count=feature_count,
                                            distance=int(distance),
                                            elapsed_time=int(elapsed_time))

    def RouteChat(self, request_iterator, context):
        prev_notes = []
        for new_note in request_iterator:
            for prev_note in prev_notes:
                if prev_note.location == new_note.location:
                    yield prev_note
            prev_notes.append(new_note)


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    route_guide_pb2_grpc.add_RouteGuideServicer_to_server(RouteGuideServicer(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    logging.basicConfig()
    serve()